<a href="https://colab.research.google.com/github/mkbahk/AmazonBraket/blob/main/AmazonBraket_backend_estimator_Xiaofan_20250314.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First, ensure that you have the following minimum package versions:
- `amazon-braket-sdk>=1.90.1`
- `qiskit-braket-provider>=0.4.3`

from qiskit_braket_provider import BraketProvider

from qiskit.primitives import BackendEstimatorV2

provider = BraketProvider()

estimator = BackendEstimatorV2(backend=provider.get_backend("SV1"))

In [ ]:
%pip install -U "amazon-braket-sdk>=1.90.1" "qiskit-braket-provider>=0.4.3"

In [ ]:
from qiskit_braket_provider import BraketProvider

provider = BraketProvider()

In [ ]:
from qiskit.circuit.library import QAOAAnsatz
from qiskit.quantum_info import SparsePauliOp

entanglement = [tuple(edge) for edge in backend.coupling_map.get_edges()]
observable = SparsePauliOp.from_sparse_list(
    [("ZZ", [i, j], 0.5) for i, j in entanglement],
    num_qubits=backend.num_qubits,
)
circuit = QAOAAnsatz(observable, reps=2)
# the circuit is parametrized, so we will define the parameter values for execution
param_values = [0.1, 0.2, 0.3, 0.4]

print(f">>> Observable: {observable.paulis}")

In [ ]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_circuit = pm.run(circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)
print(f">>> Circuit ops (ISA): {isa_circuit.count_ops()}")

In [ ]:
from qiskit.primitives import BackendEstimatorV2
estimator = BackendEstimatorV2(backend=provider.get_backend("SV1"))  # Run on SV1 simulator - replace with desired device
job = estimator.run([(isa_circuit, isa_observable, param_values)])
print(f">>> Job ID: {job.job_id()}")
print(f">>> Job Status: {job.status()}")

In [ ]:
job.result()